# **Chap 10：大语言模型 LLM**

In [ ]:
import tensorflow as tf
from keras.layers import Bidirectional, LSTM
from keras.layers import Dense, Embedding
from keras.models import Sequential
load_llm = lambda: ... # 导入 LLM 模型的逻辑
get_embedding_layer = lambda: ... # 获取嵌入层的逻辑

## **10.2 参数高效微调 PEFT**

### **10.2.1 P-Tuning相关技术（Prompt Tuning）**

在预训练大语言模型（LLM）和 Zero-Shot 概念出现后，人们就可以通过问答对话的方式与模型进行交互，并且从模型的回答中获得 NLP 任务的结果，这与传统的模型预测有非常大的不同

+ 传统模型特点：
    - 输入数据需要模型输入要求进行准备和预处理
    - 需要选定、搭建特定任务的下游网络，例如文本分类、QA、命名实体标注等
    - 模型预测的结果是特定任务的输出，例如分类任务就是输出分类标签，或者打分的概率分布

例如如下的文本分类任务：

**任务类型**：文本情感分类，`0-消极，1-积极。2-中性`  
**模型输入**："我爱大语言模型！"  
**模型输出**：1 或者 `[0.01, 0.90, 0.09]` # 输出类别，或者概率分布

+ 预训练大模型结合 Zero-Shot 特点：
    - 输入数据和需要完成的 NLP 任务由使用者组织为一段话直接提供给模型
    - 模型从输入中理解所需要完成的任务，并给出回答
    - 模型给出的结果是字符串，而有效的预测就包含在字符串中，模型的回答格式可能不统一

例如如下的文本分类任务

**任务类型**：文本情感分类  
**模型输入**："判断下面这句话的情绪是积极、消极还是中性。'我爱大语言模型！'"  
**模型输出**："积极" 或者 "这句话是积极的" 或者 "这句话表达的情绪是积极的" # 输出字符串，格式不统一

ChatGPT 开始，LLM 展现出了强大的潜力和通用能力，但是为了更好地使用 LLM，人们需要注意提问的形式，例如在上面文本分类的例子中，我们可能只想要模型回答 "积极、消极、中性" 中的某一个，而不要有过多废话，这样子利于应用到大规模文本的 NLP 任务，则模型的输入可以尝试调整为：


**任务类型**：文本情感分类  
**模型输入**："判断下面这句话的情绪。'我爱大语言模型！'，请回答'积极'，'消极'或者'中性'"  

类似的场景还有很多，在实践中人们发现<font style="color:#DF2A3F;">一个 LLM 重要的问题</font>，**组织提供给模型的提问形式对模型回答的质量影响重大，一个好的提问组织方式能够让模型回答的质量、准确性大幅提升**。这时候，**<font style="color:#DF2A3F;">Prompt</font>** 的概念被提出

+ Prompt 可以理解为自然语言构成的模板（Pattern，Template）
+ 人们会把提供给 LLM 的输入拆解为 Prompt 和 Context，Prompt 作为提供给 LLM 的指令，而 Context 是 LLM 完成指令时参考的内容，例如下面演示的 NLP 的总结任务
    - 有时候，Prompt 也可以指代提供给 LLM 的整个输入，而不对内容加以区分

```sql
任务类型：文本总结
模型输入：'总结下面的一段话，随着 ChatGPT 的发布，自然语言处理和人工智能再次引爆全社会关注，
人工智能相关和话题成为人们的饭后闲谈，投资界和金融界也将目光转向了各个大语言模型的开发和应用，
要求语言正式，字数 20 个字之内'
Prompt：'总结下面的一段话' 和 '要求语言正式，字数 20 个字之内'
Context：'随着 ChatGPT 的发布，...，各个大语言模型的开发和应用'
```


为了能够获得 LLM 更好的回答质量，一些研究者思考**为特定的任务设计合适的提问模板，即设计优质的 Prompt**，这一类技术被称为 **Pattern-Exploiting Training（PET）**，PET 通过人工构建的模版与 BERT 的掩蔽语言模型 MLM 结合，能够起到非常好的零样本、小样本以及半监督学习效果。

总结来说，Prompt 可以理解为将提供给 LLM 的任务统一为某种"完形填空类"的任务，模型根据输入的内容填补出 Prompt 所需的空缺，这些**模版属于语言模型的“探针”，我们可以通过 Prompt 来更好地抽取语言模型的特定知识，从而做到不错的 Zero Shot 效果**

+ 从 GPT-3 和 ChatGPT 的实验报告中，我们可以看到 One Shot 和 Few Shot 能大幅提高 LLM 的回答效果，这也**侧面应证了文本模板对 LLM 的帮助**，无论这个模板是出现在提问，还是出现在 Few Shot 的回答

但 **<font style="color:#DF2A3F;">PET 以及选择特定的 Prompt 不是长久之计</font>**，原因在于：

+ 对于某些任务而言，人工构建模版并不是那么容易的事情，模型的优劣我们也不好把握，而不同模型之间的效果差别可能很大（有时候，人工标注一些样本可能比构建模版还要轻松得多）
+ NLP 任务千千万，针对所有任务去设计 Prompt 不是合理的技术解决方案
+ 从 LLM 使用者的角度考虑，模型的设计也应该是去适应大家更灵活的提问，去适应更复杂的场景，以表明 LLM 具有更强的 NLP 能力，而不是反过来让用户去适应模型的提问方式，去使用固定的 Prompt

因此，如何**根据输入文本自动在模型中创建 Prompt，来提高模型的回答效果成为了一个重要的研究方向**

#### **10.2.1.1 P-Tuning（Prompt Tuning）**

**论文：GPT Understands, Too**[PtuningV1.pdf](../source/Chap10/PtuningV1.pdf)  

**方法核心：**P-Tuning 重新审视了 Prompt 的定义和意义，**<font style="color:#DF2A3F;">放弃了 “Prompt 是由自然语言构成的模板” 这种自然的想法，将 Prompt 的构建从人工构建的离散词元选择转换为连续参数的优化问题</font>**，方法简单，但成效明显

**(1) 思考**：Prompt 在 LLM 中的意义

首先思考，什么是 Prompt？

+ 表面上看，Prompt 是一些**插入到原始文本序列中的前缀 / 后缀**
+ 它们是人们特定组织好的自然语言文本，因此在模型看来，它们是输入序列中的一些词元，**Prompt 会与其他词元一样，首先经过词元嵌入层，转换为 embedding 向量**，然后是多层的 Attention
+ 通过 Prompt 模板，我们能让新的 NLP 任务与 LLM 的预训练任务一致，或者保持相似，这样才能更加充分地利用 LLM 模型，起到更好的零样本、小样本学习效果


因此，**<font style="color:#DF2A3F;">寻找合适的 Prompt 模板的过程可以抽象为一个优化问题</font>**：

+ 以语言模型 LM 为例，我们寻找插入到文本序列$ \boldsymbol{X}, \boldsymbol{Y} $（$ \boldsymbol{X} $作为 inputs，$ \boldsymbol{Y} $是标签回答）的一些前缀 / 后缀的合适的词元 $ [P_1,,\cdots,P_i], [P_{i+1},\cdots,P_m] $，得到序列$ [P_1,,\cdots,P_i], \boldsymbol{X}, [P_{i+1},\cdots,P_m], \boldsymbol{Y} $
+ 这些词元来自于模型的词典 / 词表，假设词表中的词元集合为 $ V $，则我们得到了**一个离散优化问题**：

    $$ \mathop{\min}\limits_{P_1,\cdots,P_m\in V} \mathcal{L}_{LM}\left(\mathcal{M}([P_1,\cdots,P_i],\boldsymbol{X},[P_{i+1},\cdots,P_m]), \boldsymbol{Y} \right) $$

    其中，$ \mathcal{L}_{LM} $是语言模型 LM 的损失函数


**求解上面的这个离散优化问题是困难的**，但是，即然词元形式的 Prompt （即离散形式）都会经过嵌入层转换为 embedding 向量（稠密的连续形式），我们就完全不必在乎 Prompt 是否是”自然语言“词元构成的

+ 从模型的运算逻辑上看，我们并不需要关心 Prompt 长什么样，我们关心模版由哪些词元组成
+ 但<font style="color:#DF2A3F;">**我们甚至可以跳过寻找 Prompt 词元的这一步，直接思考寻找 Prompt 词元嵌入后的 embedding 向量**</font>
+ 模板是不是自然语言不再重要，PET 等之前构造自然语言模板 Prompt 的方法，只是为了更好地保持当前推理任务和 LLM 预训练任务的一致性，但这不是必须的

因此，P-Tuning（Prompt Tuning）的<font style="color:#DF2A3F;">**核心思想**</font>就是：<font style="color:#DF2A3F;">**直接构造 Prompt 的 embedding 向量，将之前通过人工设计寻找 Prompt 的离散优化问题，转换到在连续向量空间中进行**</font>，因此 P-Tuning 可以非常简单地延续深度学习的梯度反传的框架来学习这种连续型的 Prompt 嵌入向量，所以方法称为 Prompt 微调

**(2) 方法实现**

假设我们要进行微调的模型为$ \mathcal{M} $，模型的第一层是词元嵌入层，我们记为$ \mathbf{e} $，对于词元$ X\in V $，我们将嵌入后的 embedding 向量为$ \mathbf{e}(X)\in\mathbb{R}^p $，P-Tuning 会**先选择一个模板 template**

+ **template 会包含 2 到 3 个整数**$ [s_1,s_2,s_3] $，作为 P-tuning Prompt 的分段，用于插入到原文本序列的作为前缀 / 后缀
    - **Prompt 词元的个数**$ s=s_1+s_2+s_3 $**是模型的超参数**，取决于每个超参数$ s_1,s_2,s_3 $
+ 不同类型模型 template 不太一样，例如双向 BERT 和单向 GPT 的模板 template 就不太相同
+ 假设词嵌入维度为$ p $，则构造$ s $个 Prompt 嵌入向量$ h_1,h_2,\cdots,h_s\in\mathbb{R}^p $，假设词元序列$ \boldsymbol{X}=(X_1,\cdots,X_T),\boldsymbol{Y}=(Y_1,\cdots,Y_{T'}) $，**以包含 2 个整数的 BERT 模板为例**，则**P-Tuning 的输入会按照模板 template 构造给 BERT 编码器的序列为**：

    $$ h_1,\cdots,h_{s_1},\mathbf{e}(X_1),\cdots,\mathbf{e}(X_T),h_{s_1+1},\cdots,h_{s_1+s_2},\mathbf{e}(Y_1),\cdots,\mathbf{e}(Y_{T'}) $$

如下图所示：

<img src="../source/Chap10/BERT-PTuning.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

在**训练模型时**，P-Tuning 分两种情况进行：

+ **情况一，标注的数据比较少时，做小样本微调**
    - 这种情况下，**P-Tuning 冻结 LLM 的所有参数**，包括词嵌入层的参数，只优化 Prompt 的词嵌入向量$ h_1,\cdots,h_s $，即**我们要学习**$ s $**个 embedding 向量，使得它们能够起到 Prompt 模板的作用**
    - **因为 LLM 模型的参数被冻结，这种微调方案会大大减小计算成本，训练速度较快**，并且相对于 LLM 的参数，该方案要学习的参数数量很少，大约只有整个 LLM 模型的 0.01%
+ **情况二：标注数据充足**
    - 当想要把通用的 LLM 迁移到某一个专业领域时，就会收集专业领域的大量样本来微调模型，此时如果继续使用情况一中的微调方法，**由于可以训练的参数量很少，容易出现模型欠拟合**
    - 因此，我们可以解冻 LLM 的参数，使全模型和 Prompt 嵌入向量一起进行反向传播


最后，在 P-Tuning 的原论文中，$ s $个 Prompt 词嵌入向量$ h_1,h_2,\cdots,h_s $并非直接基于随机初始化的参数进行优化，而是**做了一次重参数化（Reparameterization）**，简单来说，对 Prompt 词元做嵌入后，还**搭配了一个双向 LSTM 模型和一个全连接网络做一次加工**，才获得了最终插入到词元嵌入向量中的 Prompt 嵌入向量

+ 因此，**P-Tuning Prompt 词嵌入的步骤大致**为：
    - 首先，根据 Prompt 模板 template 生成几个**伪 token（pseudo token）**（通常按0，1，2，...，顺序编号作为 pseudo token 的词元索引）
    - 用一个嵌入层，将 pseudo token 嵌入为词向量$ h_1,h_2,\cdots,h_s $
    - 将$ h_1,h_2,\cdots,h_s $作为序列，**用一个双向 LSTM 提取每个时间步的隐状态作为新的词嵌入特征**
        $$ h_1,h_2,\cdots,h_s=\text{BiLSTM}(h_1,h_2,\cdots,h_s) $$
    - 最后，再通过一个多层感知机，对输出结果做一次变换
        $$ h_1,h_2,\cdots,h_s=\text{MLP}(h_1,h_2,\cdots,h_s) $$
    - 最后得到的$ h_1,h_2,\cdots,h_s $会按照模板插入到其他次元的嵌入序列$ \mathbf{e}(\boldsymbol{X}),\mathbf{e}(\boldsymbol{Y}) $中

作者认为这样设计和构造 Prompt 词嵌入向量的优点在于：

+ LSTM 处理后**能让 Prompt 词元的表示在序列上相关性更强**，几个 Prompt 词元不再是独立的，这在某种程度上更像“自然语言”，因为自然语言的词元之间不是独立的
+ 此外，在实践中发现，加入 LSTM 结构能够防止局部最优，效果上可以**使模型收敛更快**

我们来看**核心代码的实现**

+ **Prompt 模板的词嵌入编码器**

In [ ]:
class PromptEncoder(tf.keras.layers.Layer):
    def __init__(self, template : tuple, embed_size : int, **kwargs):
        super(PromptEncoder, self).__init__(**kwargs)
        self.num_tokens = sum(template) # Prompt 的 token 數量
        self.embed_size = embed_size # 词嵌入维度
        # Prompt 的嵌入层
        self.embedding = Embedding(self.num_tokens, self.embed_size)
        # 双向 LSTM 层加工嵌入向量
        # LSTM 输出维度 embed_size // 2，双向模型会使得维度翻倍
        self.lstm = Sequential([
            Bidirectional(LSTM(self.embed_size // 2,return_sequences=True)),
            Bidirectional(LSTM(self.embed_size // 2,return_sequences=True)),
        ])
        # MLP 层加工 LSTM 输出
        self.mlp = Sequential([
            Dense(self.embed_size, activation='relu'),
            Dense(self.embed_size),
        ])
        
        # 生成 pseudo token
        self.pseudo_token = tf.range(
            self.num_tokens, dtype=tf.int32)[tf.newaxis, :] # [1, num_tokens]

    def call(self, **kwargs):
        # 形状：[1, num_tokens, embed_size]
        input_embeds = self.embedding(self.pseudo_token) 
        # 访问下标 [0] 去掉批量维度，形状：[num_tokens, embed_size]
        lstm_embeds = self.lstm(input_embeds,**kwargs)[0]
        # mlp 再做一次投影，不改变形状
        output_embeds = self.mlp(lstm_embeds,**kwargs)
        return output_embeds

编码器接收一个模板参数 template，它是一个元组，由整数组成，例如 template = (3,3)，编码器的输出是一个张量，形状为$ (s,p) $，其中$ s=\sum_i \text{template}[i] $表示 Prompt 模板词元个数，$ p $是 LLM 的词嵌入维度

In [ ]:
# 提供一个模板 template
prompt_encoder = PromptEncoder(template=(3,3), embed_size=64)
prompt_encoder.call(training=False).shape # TensorShape([6, 64])

+ **将 Prompt 词嵌入插入到词元序列中作为前缀 / 后缀**

**基于 LLM 模型，我们可以构造**`PromptTuning`类，下面以 P-Tuning 中的 GPT 模型为例，类`PromptTuning`的函数`get_queries()`用于准备好输入给 LLM 编码器的词嵌入向量

In [ ]:
class PromptTuning(tf.keras.Model):
    def __init__(self, template, *args, **kwargs):
        super(PromptTuning, self).__init__(*args, **kwargs)
        self.llm = load_llm() # 加载预训练的语言模型
        # 获取 LLM 词嵌入层
        self.embedding = get_embedding_layer(self.llm)
        # 获得 LLM 的词嵌入维度
        self.embed_size = self.embedding.get_config()["output_dim"]
        # Prompt 编码器
        self.template = template
        self.prompt_encoder = PromptEncoder(template, self.embed_size)

    def get_queries(self, x_hs, x_ts=None, **kwargs):
        # x_hs：第一句话的 token id，形状为 [batch, seq1_len]
        # x_ts：第二句话的 token id，形状为 [batch, seq2_len]，GPT 模型为 None

        batch = tf.shape(x_hs)[0] # batch 大小
        # 计算 Prompt 编码器向量
        prompt_embeds = self.prompt_encoder.call(**kwargs) # [s, embed_size]
        # 添加批量维度并重复，形状：[batch, s, embed_size]
        prompt_embeds = tf.repeat(
            prompt_embeds[tf.newaxis, :, :], batch, axis=0) 

        # 以 GPT 模型为例
        if self.llm.config.model_type == "gpt":
            # 获取 LLM 词嵌入向量
            input_embeds = self.embedding(x_hs) # [batch, seq1_len, embed_size]
            s1, s2, *_ = self.template
            # 将 Prompt 编码器向量与 LLM 词嵌入向量拼接
            input_embeds = tf.concat([
                prompt_embeds[:, :s1, :],
                input_embeds,
                prompt_embeds[:, s1:(s1+s2), :],    
            ], axis=1) # [batch, s1 + seq1_len + s2, embed_size]

            return input_embeds
        else:
            raise NotImplementedError("The query template has not been defined.")

    def call(self, x_hs, x_ts=None, **kwargs):
        # 模型的推理逻辑，调用 get_queries() 获得插入 Prompt 嵌入后的输入 input_embeds
        # 再将 input_embeds 送入 LLM 的后续计算层中，类似于
        input_embeds = self.get_queries(x_hs, x_ts, **kwargs)
        output = self.llm.encoder(input_embeds) # 调用 LLM 的编码器
        ...

对比人工构造 Prompt 类方法，结合上面的部分核心代码，现在理解 Prompt Tuning 的结构和思想会更清晰，**Prompt Tuning 连续型向量的处理思路与离散优化对比如下图所示**：

<img src="../source/Chap10/PTuning示意图.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

**(3) P-Tuning 效果和思考**

P-Tuning 的各项数据集对比实验中有两个重要的结论：

+ 在部分任务上，通过这种 **P-Tuning 微调方式能够达到甚至超过 LLM 全参数微调（FineTuning）的性能**，P-Tuning 配合 BERT，GPT 等模型都能提升性能，说明 P-tuning 对 LLM 潜能的释放是通用的
+ 对比 GPT-base 和 BERT-base，在大部分数据集上，**基于 P-Tuning，GPT-base 模型都取得了更好的结果，这个实验颠覆了人们之前的认知**，在之前，人们都认为在自然语言理解任务上，BERT 为代表的双向编码器模型要优于 GPT 为代表的单向编码器，但 P-Tuning 展现出了 GPT 作为单向编码器模型的强大强力

总结来说，**P-Tuning 对 LLM 带来的提升包括**：

+ 从小样本视角来看，P-Tuning 取得了最优的小样本学习效果
+ P-Tuning 应用简单，对于任何 LLM，我们**只需要更改模型的输入部分**，中间编码器的结构不需要改变
+ P-Tuning 冻结 LLM 参数只调整 Prompt 嵌入向量，使得**优化的参数量大概只是 LLM 的 0.01%，这大大减少了微调 LLM 的成本**
+ 从 Prompt 模板构建的角度，P-Tuning 比人工构建的模板类方法（PET）效果要好很多，也省力得多，**将 Prompt 模板寻找的建模问题从离散空间转换到连续空间**，从而可以借助深度学习的反向传播框架快速部署
    - **Prompt Tuning 不是第一个提出这种优化思想的，在此之前有 Prefix Tuning**，就提出了这种优化连续型嵌入向量的概念
+ 从模型角度，P-Tuning 帮助同规模参数的 GPT 性能释放到与 BERT 相近，揭示了单向编码器模型 GPT 也具有很强的自然语言理解能力


但**P-Tuning 也存在一些明显的缺陷**

+ 对不同类型的模型（单向编码器 GPT，双向编码器 BERT），需要选择不同的 Prompt 插入模版，**Prompt 模板不是通用的**
+ 模型的**可训练参数只包含在一开始的输入层**，这有两个问题：
    - 可训练的参数量有限，对于困难复杂的 NLP 任务，**参数量不足可能导致欠拟合**
    - 对于 LLM，在**输入和输出之间还间隔了很多层 Attention，这使得我们很难预估输入的改变对模型输出的影响大小**，对深层网络而言，输入的改动可能对模型预测的影响很有限
+ 构造 Prompt 嵌入向量时所使用的 LSTM 结构**缺乏合理的归纳偏置假设**

#### **10.2.1.2 P-Tuning V2（Prefix Tuning & Deep Promtp Tuning）**

**论文：P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks**[PtuningV2.pdf](../source/Chap10/PtuningV2.pdf)  

**<font style="color:#DF2A3F;">方法核心</font>**：P-Tuning V2 **结合了 Prefix Tuning，Prompt Tuning 和 Deep Prompt Tuning 的核心思想，将 Prompt 嵌入向量统一插入到文本序列前段，并插入到 LLM 的每一层中**，同时 P-Tuning V2 **去掉了 Prompt Tuning 中的重参数化过程**，实验表明重参数化对广泛任务而言是不必要的

**(1) 对 Prompt Tuning 的思考**

有了 Prompt Tuning 的成功，P-Tuning V2 从以下几个方面进行了思考，并做出了改变

从模型结构的考虑上，P-Tuning 需要根据不同类型的 LLM 设计不一样的 template

+ 这种**缺乏统一通用性的设计缺乏模型设计的美感**，同时增加了工作量
+ 从 LLM 模型结构出发，所有的 **LLM 文本编码器都是基于 Attention 架构，而注意力机制的计算特点使得模型对 Prompt 的插入位置其实并不是很明感**，如果不考虑 PositionalEmbedding 层，Prompt 插入到任何位置对于 Attention 而言都是等价的

因此 P-Tuning V2 **采用 Prefix Tuning 的方法**，**认为根据不同模型设计不同的 Prompt template，然后插入到原文本序列的做法是不必要的**，因此 V2 将 Prompt 嵌入向量插入到文本序列的前端作为前缀

另外一方面，V2 作者观察到 **P-Tuning 在一些复杂困难的自然语言理解任务上（命名实体识别，从文本中抽取QA）表现较差（远差于全参数 FineTuning）**，V2 认为 P-Tuning 仅调整输入层的 Prompt 嵌入向量带来了两个问题：

+ 受限于输入序列的长度限制，P-Tuning 可训练的参数是很有限的，这在困难任务微调中会**导致模型欠拟合**
+ **输入层 embedding 的改变对模型的预测是间接的**，尤其对于深层的 LLM，我们无法评估输入层的改变多大程度上会对模型预测有作用

因此 V2 **采用了 Deep Prompt Tuning 的思想**，**将 Prompt 嵌入向量从输入层扩展到每一层，并将其插入到 LLM 注意力编码器的每一层序列的前端**
+ 一方面，**V2 有了更多的可训练参数（从 P-Tuning 的 0.01% 增加到 0.1% ～ 3%）**，这在保留 P-Tuning 微调成本低的优点的同时，让模型在特定任务有更多的参数可供调整
+ 另一方面，**V2 的新结构使得 Prompt 嵌入向量能够更加直接地作用到模型预测**（因为 Prompt 嵌入向量现在存在于 LLM 编码器的每一层）

P-Tuning 和 P-Tuning V2 的对比图如下所示，在图中，**蓝色块儿是被冻结的 LLM 模型，而橙色块儿是两种方法可以被训练的模型参数**，从图中能清晰地明白 V2 中使用 Deep Prompt Tuning 中 Deep 的含义

<img src="../source/Chap10/PTuningV2示意图.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

最后，V2 指出，类似于 P-Tuning 中**对 Prompt 嵌入向量做 LSTM 和 MLP 变换的这种重参数化可能是不必要的**，重参数化在不同任务、不同数据集上的表现有差异，并不能保证使用了重参数化的 Prompt 嵌入向量能取得更好的效果，**有时候使用重参数化甚至会给模型性能带来负面影响**

+ 这在前面的介绍中也说明过，**重参数化的设计缺乏一个合理的归纳偏置假设**
+ 因此，**V2 将重参数化设定为 Optional，默认情况下不使用重参数化**

**(2) 方法实现**

一些数据上的表示在前面的 P-Tuning 已经有了详细的介绍，V2 与其没有本质的区别，**V2 去掉了模板参数 template，且只有一个超参数，Prompt 长度**`prompt_len`，在方法的实现上，核心区别在于下面两点**：

+ Prompt 嵌入向量只添加到词元序列的前端，作为前缀
+ Prompt 嵌入向量需要插入到 LLM 编码器的每一层

我们还是从 `PromptEncoder`和插入到 LLM 模型推理`PromptTuningV2`两个类的实现来对比 V2 的改变

+ **V2 Prompt 模板的词嵌入编码器**

In [ ]:
class PromptEncoderV2(tf.keras.layers.Layer):
    def __init__(self, prompt_len : int, config : dict, 
                 reparam : bool=False, **kwargs):
        super(PromptEncoderV2, self).__init__(**kwargs)
        # Prompt 长度和 LLM 的词嵌入维度
        # config : 存储 LLM 配置信息的字典，例如保存词嵌入维度，模型层数等
        self.prompt_len = prompt_len
        self.embed_size = config["hidden_size"]
        self.num_layers = config["num_layers"]
        self.use_reparameterization = reparam

        # 生成 Prompt 的 pseudo token
        self.pseudo_token = tf.range(
            self.prompt_len, dtype=tf.int32)[tf.newaxis, :] # [1, num_tokens]
        
        """
        注意无论是否使用 Reparameterization
        最后输出的维度都是 hidden_size * num_layers * 2
        我们为每一层生成的特征维度是两倍的 hidden_size，即 2 * hidden_size
        因为在传递给 LLM 的注意力编码器时，它们会一分为二作为注意力的 key 和 value
        此时，key 和 value 的特征维度正好是 hidden_size
        """
        # 使用 Reparameterization，需要做 MLP 投影
        if self.use_reparameterization:
            self.embedding = Embedding(self.prompt_len, self.embed_size)
            self.projecitor = Sequential([
                Dense(self.embed_size, activation='tanh'),
                Dense(self.embed_size * self.num_layers * 2),
            ])
        # 不使用 Reparameterization，直接使用 Embedding 层
        else:
            self.embedding = Embedding(
                self.prompt_len, self.embed_size * self.num_layers * 2)

    def call(self, **kwargs):
        # prompt_embeds形状：[prompt_len, hidden_size * num_layers * 2]
        if self.use_reparameterization:
            prompt_embeds = self.projecitor(
                self.embedding(self.pseudo_token)[0],**kwargs)
        else:
            prompt_embeds = self.embedding(self.pseudo_token)[0]

        return prompt_embeds


+ **V2 将 Prompt 词嵌入插入到词元序列的前缀，并且插入到每一层**

函数`get_prompts`会为 LLM 的计算准备好 Prompts 嵌入向量，准备工作主要包括：

+ 将生成的`prompt_embeds`添加批量维度，并重复`batch_size`次
+ **将其变换形状到 Attention 计算中多头注意力所需的形状**

    `[batch, seq_len, num_heads, hidden_size // num_heads]`

+ 注意，我们**在生成 Prompt 嵌入向量时，特征维度设置为**`2 * hidden_size`，这是**因为 LLM 注意力层计算时需要用到 Prompt 作为**`key`和`value`，通过`tf.split()`将 Prompt 嵌入向量一分为二，分别作为注意力中的`key`和`value`使用，则它们的特征维度都恢复到`hidden_size`

In [ ]:
class PromptTuningV2(tf.keras.Model):
    def __init__(self, prompt_len : int, *args, **kwargs):
        super(PromptTuningV2, self).__init__(*args, **kwargs)
        self.llm = load_llm() # 加载预训练的语言模型
        self.prompt_len = prompt_len
        self.prompt_encoder = PromptEncoderV2(prompt_len, self.llm.config)
    
    # 用于准备提供给 LLM 编码器的 Prompt 的函数
    def get_prompts(self, batch : int, **kwargs):
        # 生成 prompt 嵌入向量，形状：[prompt_len, hidden * num_layers * 2]
        prompt_embeds = self.prompt_encoder.call(**kwargs)

        # 添加批量维度并重复，形状：[batch, prompt_len, hidden_size * num_layers * 2]
        prompt_embeds = tf.repeat(
            prompt_embeds[tf.newaxis, :, :], batch, axis=0)
        
        # 变换形状为 
        # [batch, prompt_len, num_layers * 2, num_heads, hidden_size // num_heads]
        prompt_embeds = tf.reshape(
            prompt_embeds, 
            [batch, 
             self.prompt_len, 
             self.llm.config["num_layers"]*2, 
             self.llm.config["num_heads"], 
             self.llm.config["hidden_size"] // self.llm.config["num_heads"]])
        
        # 交换维度，形状：
        # [num_layers * 2, batch, prompt_len, num_heads, hidden_size // num_heads]
        prompt_embeds = tf.transpose(prompt_embeds, [2, 0, 1, 3, 4])
        # 将第一个维度（num_layers * 2）分裂，分别作为 Attention 的 key 和 value
        # prompt_embeds 是包含两个元素的列表，分别是 key 和 value
        # key, value 形状：
        # [num_layers, batch, prompt_len, num_heads, hidden_size // num_heads]
        prompt_embeds = tf.split(prompt_embeds, 2, axis=0)

        return prompt_embeds

    def call(self, inputs, **kwargs):
        # 准备提供给 LLM 编码器的 Prompt
        batch_size = tf.shape(inputs)[0]
        prompt_embeds = self.get_prompts(batch_size, **kwargs)

        # 调用 LLM 编码器，在每一层上迭代
        for i, layer in enumerate(self.llm.encoder.layers):
            ... # 前置逻辑

            # 当前层的 query, key 和 value
            # key, value 形状: 
            # [batch, seq_len, num_heads, hidden_size // num_heads]
            query, key, value = ...

            # 将 Prompt 作为前缀插入，形状变为
            # [batch, prompt_len + seq_len, num_heads, hidden_size // num_heads]
            key_with_prompt = tf.concat([prompt_embeds[0][i], key], axis=1)
            value_with_prompt = tf.concat([prompt_embeds[1][i], value], axis=1)

            # 调用当前层的 Attention 进行计算
            output = layer.attention(
                query=query, 
                key=key_with_prompt, 
                value=value_with_prompt,**kwargs)

            ... # 后续逻辑

**(3) P-Tuning V2 效果和思考**

作者通过 P-Tuning V2 的实验得到了两个重要的结论：

+ **<Prompt 长度在 V2 中很重要**：P-Tuning V2 中 Prompt 长度`prompt_len`作为方法的一个超参数，会显著影响模型微调的性能表现
    - **在一些简单的文本分类任务中，模型偏向于一个较小的**`prompt_len`，通常小于 20
    - **而在复杂的自然语言理解任务中，例如句子标记，偏向于较长的**`prompt_len`，约为100
+ **多任务学习有助于提升 V2 微调模型的泛化能力**：在使用 P-Tuning V2 微调 LLM 时，如果微调任务包含多种 NLP 任务，则微调效果，尤其是语言模型的泛化能力能由于单目标任务

此外，很重要的一点是，**P-Tuning V2 使得这种轻量微调 LLM 的方法能够在更多的、更复杂的任务上取得和全参数 FineTuning 一样的结果，并且在 Zero Shot 的模型泛化性任务上，P-Tuning V2 甚至还要超过 FineTuning**，对 P-Tuning V2 和 FineTuning 进行详细实验的论文：**PE Prompt Tuning Makes Generalized and Calibrated Neural Text Retrievers**[Parameter-Efficient Prompt Tuning Makes Generalized and Calibrated Neural Text Retrievers.pdf](../source/Chap10/Parameter-Efficient%20Prompt%20Tuning%20Makes%20Generalized%20and%20Calibrated%20Neural%20Text%20Retrievers.pdf)

作者在实验中发现 P-Tuning V2 微调的模型相比全参数 FineTuning 在泛化性上具有更好的表现，然后**从模型一致性上出发，尝试从理论上解释该现象，作者定义了一个模型预测与真实标签的一致性指标，Prompt Tuning 在各种任务、各种数据集上都有更好的一致性**，如下图所示：

<img src="../source/Chap10/PTuningV2一致性.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

此外，对比不同 Quer-Length 下模型的表现，发现：

+ **P-Tuning V2 具有更好的 Query-Length 鲁棒性**，在两个经典数据集上，随着 Query-Length 改变，尤其是当 Query-Length 极端小 / 极端大时，FineTuning 的性能与 P-Tuning V2 的差距会加大
+ 作者认为，在**多样化的 NLP 任务中，输入给模型的 Query-Length 变化范围很大**，有的任务文本，其提问部分的词元长度可能只有 8～40 个词元，而有些数据集中 query 的长度可能会很长
+ 而**全参数 FIneTuning 会调整 LLM 的位置嵌入层`PositionalEmbedding`，这可能会**使得 LLM 定位和提取 NLP 任务存在模型偏差**，从而多任务的表现下滑，因为特定类型的任务具有某种类型的 Query-Length
+ 但 **Prompt Tuning 类型的微调方法，LLM 的参数全部被冻结，`PositionalEmbedding`层不会被调整**，因此在问题定位，和多任务语言理解上有更好的泛化能力

<img src="../source/Chap10/PTuningV2对比querylength.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

----

### **10.2.2 LoRA（Low-Rank Adaptation）**

### **10.2.2.1 技术背景**

在微调 LLM 使其更适用于专业领域的下游任务时，一个很大的问题是**随着 LLM 的参数规模增长，微调训练模型所需要的硬件成本极高，甚至在一般的 GPU 上不可行**，为此人们提出了很多**低参数化的高效微调方案，称为 PE （Parameter-Efficient）**，核心思想都是尽可能地减少 LLM 微调训练时的可训练参数量，主流方法包括：

+ **Adapter**：在**每个 Transformer 的注意力层之间插入一个小的 MLP 模块**，模块的参数量相对整个 LLM 而言占比很小，微调时只调整 MLP 模块的参数
+ **BitFit**：微调时**只调整 LLM 所有的偏置项 bias**，而冻结所有的权重项 weights
+ **Prompt Tuning**：代表方法包括 Prefix-Tuning，P-Tuning，Deep Prompt Tuning 和 P-Tuning V2，**通过在输入序列（或在 Transformer 的每一层输入）中插入可训练的 Prompt 词嵌入向量**来优化模型

效果比较好的 Adapter 和 Prompt-Tuning 作为微调 LLM 的两类主流方法存在一些**明显的缺陷**：

+ **对于 Adapter 而言**，由于我们在模型结构中插入了额外的层，整个模型的层数变深了，这**会给模型带来额外的推理延迟**，尤其在小批量（例如用户实时查询，`batch_size = 1`），推理延迟相比去掉 Adapter 模块显著增加
+ **对于 Prompt Tuning 而言**，这种在输入序列中插入可训练参数的方法，在训练中表现出一些问题：
    - 模型训练优化有困难，表现不太稳定，**模型的性能不会随着可训练参数的增加一致地变化**
    - 由于模型输入序列的长度是固定的，**Prompt 的插入会减少任务文本可占用的有效长度**，这对一些特定的下游任务不太友好，尤其是当 Prompt 的长度变长（增加模型的可训练参数时）

因此，LoRA 还是想**像传统微调的思路一样，直接调整 LLM 的参数，因为这不会给模型引入任何推理延迟，也不存在 Prompt Tuning 中的有效输入占用的问题**，从数学表示上，假设 LLM 预训练得到的参数为 $ \Phi_0 $，我们希望通过微调更新它为$ \Phi_0 + \Delta \Phi $，**语言模型的全参数 FineTuning 通过优化下面的目标函数实现**：
    $$ \mathop{\max}\limits_{\Delta\Phi} \sum_{(x,y)\in \mathcal{Z}} \sum_{t=1}^{|y|} \log\left( P_{\Phi_0 + \Delta \Phi}(y_t | x,y_{<t})  \right) $$

对于 FineTuning 而言，任何一个下游任务的微调，我们都会得到一个不同的更新量 $ \Delta \Phi $，并且**更新量**$ \Delta \Phi $**的规模**$ |\Delta \Phi| = |\Phi_0| $，对于像 GPT-3 这样的超大规模 LLM，训练和部署模型将会需要大量的硬件资源来完成$ \Delta \Phi, \Phi $的保存和计算

但如何实现低参数化调整呢，针对不同的下游任务，LoRA 的设计是**将更新量**$ \Delta \Phi $**记为**$ \Delta \Phi=\Delta\Phi(\Theta) $**，即将更新量用一个小的多的参数集合**$ \Theta $**来编码，使得**$ |\Theta| \ll |\Phi_0| $，此时优化问题写为：
$$ \mathop{\max}\limits_{\Theta} \sum_{(x,y)\in \mathcal{Z}} \sum_{t=1}^{|y|} \log\left( 
P_{\Phi_0 + \Delta \Phi(\Theta)}(y_t | x,y_{<t})  \right) $$

而 LoRA 具体实现上述想法时，受到 Aghajanyan 2020 的启发，**Aghajanyan 展示了当适应特定任务时，预训练的语言模型具有较低的“内在维度”（instrisic dimension），尽管将预训练模型随机投影到较小的子空间，它仍然可以有效地学习下游任务**  
论文：**内在维度解释解释语言模型微调的有效性**[Intrinsic Dimensionality Explains The Effectiveness of Language Model Fine-Tuning.pdf](../source/Chap10/Intrinsic%20Dimensionality%20Explains%20The%20Effectiveness%20of%20Language%20Model%20Fine-Tuning.pdf)

**<font style="color:#DF2A3F;">这一观点意味着，用一个极少参数量的</font>**$ \Theta $**<font style="color:#DF2A3F;">来编码</font>**$ \Delta\Phi $**<font style="color:#DF2A3F;">在理论上是完全可行的</font>**，因为微调 LLM 并不需要那么多的信息，接下来的任务就只是**如何设计这一套编码建模方案了**，LoRA 在**技术路线上选择了低秩矩阵分解**


### **10.2.2.2 LoRA：Low-Rank Adaptation**

**(1) LoRA 原理和设计**

一个神经网络通常包含许多全连接层（Dense Layer），**每个全连接层包含权重矩阵**$ W $**负责执行矩阵乘法**，权重$ W $通常都是具有满秩结构的稠密矩阵，**受 Aghajanyan 提出的内在维度的启发，LoRA 假设在微调时，权重**$ W $**的更新量**$ \Delta W $**同样具有一个较低的内在维度**

**描述一个矩阵的信息量，即矩阵的本质维度可以用矩阵的秩来表示**，**LoRA 将权重较低的内在维度表示为矩阵具有较低的秩**，对于一个预训练 LLM 的权重矩阵 $ W_0\in\mathbb{R}^{d\times k} $，**<LoRA 用矩阵的低秩分解表示来构造参数的更新量**：
    $$ W_0 + \Delta W = W_0 + BA,\quad B\in\mathbb{R}^{d\times r},A\in\mathbb{R}^{r\times k} $$

其中，矩阵$ \Delta W $的秩$ r $满足$ r\ll \min\{d,k\} $

在训练时，**参数**$ W_0 $**被冻结不会记录梯度信息进行更新，而分解矩阵**$ A,B $**是 LoRA 的可训练参数**
+ 注意$ W,\Delta W=BA $具有相同的形状，它们会乘以相同的输入，然后各自计算的结果向量会相加在一起
+ 对于某个$ h=W_0 x $，使用 LoRA 调整后的前向推理会变为：$ h=W_0x+\Delta Wx=W_0 x + BAx $

下图直观地表示了 LoRA 的示意图：

<img src="../source/Chap10/LoRA示意图.png" width=250 style="display: block; margin-left: auto; margin-right: auto;">

在使用 LoRA 时，**<font style="color:#DF2A3F;">更新参数的秩</font>**$ r $**<font style="color:#DF2A3F;">作为超参数可以进行调整，它同时控制了 LoRA 微调时的可训练参数量</font>**，当取$ r=1 $时，我们仅需极少的参数量就可以实现 LLM 微调（**<font style="color:#DF2A3F;">参数可以少到 LLM 参数量的 0.01%</font>**）

+ **微调开始时，权重**$ B $**被初始化为零矩阵，权重**$ A $**用一个高斯分布做初始化**
+ 因此，在训练的初始时刻，更新量$ \Delta W=BA=0 $


**LoRA 的微调可以看作是 FineTuning 的推广**，如果我们将 LoRA 应用到 LLM 的所有权重参数，并同时让 LLM 的偏置项参与微调，**当 LoRA 微调的秩参数**$ r $**设置为等于 LLM 参数**$ W_0 $**的秩时，LoRA 便大幅增加了微调时的可训练参数，训练 LoRA 基本上等同于在原始的 LLM 上做 FineTuning**

在部署 LoRA 微调部署时，我们可以直接加载 LoRA 的参数$ A,B $然后更新 LLM 的参数$ W=W_0 + BA $，这个部署的加法过程只需要在推理前花费一小会儿完成，**加载之后模型推理的过程与原始的 LLM 没有任何区别，所以 LoRA 相比 Adapter 不会存在任何推理延迟**

+ 而如果我们想**将模型切换到另外一种下游任务，切换到另外一种领域，我们只需要恢复参数**$ W_0 $**，然后在此基础上加上另一套任务的参数**$ B'A' $，该切换操作只需要极小的内存消耗

LoRA 的微调思路可以应用到任何神经网络的稠密层，当**把 LoRA 部署到 Transformer 中时**，Transformer 的自注意力架构中有四组权重参数$ W_q,W_k,W_v,W_o $分别对应 Query，Key，Value 的变换权重和输出层的变换权重，在注意力层后的 MLP 中还有两个权重参数

通常而言，自注意力层的输入输出维度保持不变，以便于实现多层堆叠，自注意力模块中的权重都是$ d_{h}\times d_{h} $的矩阵，LoRA 微调通常只调整注意力头中的这些权重，而保持 MLP 模块中的权重不变

+ 因此**LoRA 微调框架是非常灵活的，人们可以选择模型中需要调整的模块，需要调整的层，以实现各种程度的低参数化微调**
+ **即使只调整自注意力模块中的参数，LoRA 也有选择**，例如四组权重参数$ W_q,W_k,W_v,W_o $，我们可以同时调整，也可以只调整部分，例如$ W_k,W_v $，LoRA 的实现发现，**调整不同部分的参数，模型性能有明显不同**

**(2) LoRA 效果和优势**

总结来说，LoRA 可以在实验中的几个数据集上都达到甚至超过 FineTuning，并且从实验中有两个重要的结论：

+ **并非所有的微调方法所取得的模型性能会随着可训练参数的增加而增长**，相反，Prompt Tuning 容易出现随着可训练参数的增加，模型性能出现严重的下滑
+ **随着 LoRA 可训练参数数量的增加（即增加 LoRA 的秩参数**$ r $**），LoRA 微调的模型性能几乎不发生变化**，这说明两点：
    - **<font style="color:#DF2A3F;">LoRA 对超参数秩</font>**$ r $**<font style="color:#DF2A3F;">不敏感</font>**，LoRA 微调不会因为可训练参数的增加导致模型下滑
    - **<font style="color:#DF2A3F;">从特征子空间的角度考虑，这预示着参数的有效更新量</font>**$ \Delta W $**<font style="color:#DF2A3F;">有着非常低秩的结构，即</font>**$ \Delta W $**<font style="color:#DF2A3F;">的内在维度很小</font>**

<img src="../source/Chap10/LoRA微调效果随参数量的变化.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

**总结 LoRA 带来的优点**，包括：

+ LoRA 可以非常**高效地微调 LLM，并为不同的下游任务保存不同的更新参数快照**$ B',A' $，由于所调整的参数数量少，**<font style="color:#117CEE;">参数快照的存储成本和模型切换的成本都很低</font>**
+ LoRA 微调模型后，会直接更新原始 LLM 模型的参数，而**<font style="color:#117CEE;">不改变任何 LLM 的计算逻辑，因此 LoRA 不会为 LLM 的推理引入任何推理延迟</font>**
+ LoRA **<font style="color:#117CEE;">相对于之前提出的一些 PE 微调方法（Adapter，Prompt Tuning）是正交的（或者理解为提升模型的方向是无关的），因此 LoRA 可以与它们中的任何一种组合使用共同微调模型</font>**，这可能会带来更好的结果


**(3) LoRA 思考：理解 LLM 的低秩结构**

LoRA 除了为人们提供了一种新的低参数化的微调方案，更重要的是，**它为人们揭示了 LLM 中存在的内在低秩结构，低秩更新和微调 LLM 之间的关系，以及微调 LLM 到底在微调什么**

LoRA 通过一系列实验尝试解答下面的**三个问题**：

1. 对于给定的微调参数量预算，对于 Transformer LLM，微调哪部分参数能获得最优的结果？
2. 最优更新量$ \Delta W $确实是低秩的吗？如果是，在实际中我们应该选择多大的秩参数$ r $
3. 更新量$ \Delta W $和 LLM 的原参数$ W $有什么关系？$ \Delta W $会与参数$ W $强相关吗？微调到底在调什么

+ **问题 1：使用 LoRA 时，应该调整 Transformer 的哪些权重**

考虑 Transformer 自注意力模块中的四种参数$ W_q,W_k,W_v,W_o $，然后我们**限定 LoRA 可以微调的参数量，这通过限定秩**$ r $**实现**，让后将$ r $**分配到不同的参数组中**，作者取$ r=8 $，并尝试将$ r=8 $**的参数预算全部分配到同一组参数，或者它们的一些组合上**，模型性能如下表所示：


<img src="../source/Chap10/LoRA微调不同部分的参数.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

实验可以得出的结论有：

+ 将所有的微调运算给$ W_q,W_k $会得到显著更差的结果，同时调整$ W_q,W_o $能得到最好的结果
+ 限制<font style="color:#2F4BDA;">秩参数</font>$ r=4 $**<font style="color:#2F4BDA;">也足够</font>**$ \Delta W $**<font style="color:#2F4BDA;">捕捉到足够的信息</font>**，获得最好的模型性能
+ **<font style="color:#2F4BDA;">同时调整多个权重，并使用一个较低的秩</font>**$ r=2 $**<font style="color:#2F4BDA;">，比单独调整一个权重而使用一个较大的秩要好</font>**

+ **问题 2：LoRA 微调的最优的秩**$ r $**是多少**

同样考虑几种不同的微调参数组合，同时观察给定不同秩参数$ r $时 LoRA 微调的性能表现，如下表：

<img src="../source/Chap10/LoRA调整微调时的秩.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">


从结果来看，**<font style="color:#DF2A3F;">LoRA 只需要一个非常小的秩</font>**$ r $**<font style="color:#DF2A3F;">就能够取得非常有竞争力的性能，这预示着更新量</font>**$ \Delta W $**<font style="color:#DF2A3F;">拥有一个非常小的内在维度</font>**

为了论证结论的真实性，我们可以**检查不同秩参数**$ r $**下学习得到的参数的子空间的相交情况**，作者认为**当我们增加**$ r $**<时，所学习得到的参数**$ A,B $**<并不会增加新的有意义的子空间，从而采用低秩**$ r $**进行 LoRA 微调已经足够**

当给定两组秩参数$ r_1\ne r_2 $，**假设**$ A_{r_1},A_{r_2} $**分别是 LoRA 微调得到的 LLM 某一层中某个权重的参数矩阵**$ A $（分析以$ \Delta W=BA $中的$ A $为例，对矩阵$ B $完全可以进行相同的分析），它们基于相同的预训练 LLM 的初始参数微调得到，**对**$ A_{r_1},A_{r_2} $**进行奇异值分解**$ A=V^TDU $**，然后我们取右侧的奇异酉矩阵**$ U_{r_1},U_{r_2} $（取左侧的矩阵$ V $分析也是同理的，结论不会有区别）

我们将**观察**$ U_{r_1} $**中的前**$ i(i\leq r_1) $**个奇异向量张成的子空间**$ S_{r_1}(i) $**有多少与**$ U_{r_2} $**中的前**$ j(j\leq r_2) $**个奇异向量张成的子空间**$ S_{r_2}(j) $**相交**，我们通过量化地方法来度量这种相关性，**基于 Grassmann 距离定义标准化后的子空间相似度**：
    $$ \phi(A_{r_1},A_{r_2},i,j)=\frac{\| {U_{r_1}^{i}}^T U_{r_2} \|_F^2}{\min(i,j)} \in [0,1] $$
其中，$ U_r^{i} $**表示酉矩阵**$ U_r $**的前**$ i $**列，它对应与奇异值排序后的前**$ i $**个奇异向量**，度量$ \phi\in [0,1] $，当$ \phi=1 $时说明两个子空间完全相交，而$ \phi=0 $时说明两个子空间完全分离

作者取$ r_1=8,r_2=64 $，下图是子空间相似度$ \phi $随着$ i,j $的变化情况：

<img src="../source/Chap10/LoRA不同秩微调子空间正交情况.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

图中可以观察到：

+ 两个矩阵$ A_{r_1},A_{r_2} $**的头部奇异向量所对应的方向很大程度上重叠了，而对于非头部的奇异向量没有相交**
+ 特别地，无论是$ \Delta W_q $还是$ \Delta W_v $，$ A_{r_1},A_{r_2} $**共享了一个维度为 1 子空间（即**$ i=j=1 $**），其标准化后的子空间相似度大于 0.5，这解释了为什么在 LoRA 微调时**$ r=1 $**能表现如此之好**
+ $ r=8 $头部奇异向量的方向包含在了$ r=64 $中，反之亦然

由于$ A_{r_1},A_{r_2} $使用相同的预训练参数微调得到，上图指出，**微调参数**$ A_{r_1},A_{r_2} $**中头部的奇异向量的方向是对微调最有效的，而其他方向可能大部分包含的是在训练过程中累积的随机噪声**，因此更新量$ \Delta W $确实可以是一个非常低秩的矩阵，它具有非常小的内在维度


最后，作者为了确认这一结论，固定$ r=64 $，然后使用不同的随机种子进行了两次实验，对比两次微调得到的参数$ A $的子空间相似度，如下图所示：

<img src="../source/Chap10/LoRA对比两次微调时子空间相似性.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

图中可以看到：

+ **<font style="color:#117CEE;">两次实验虽然随机种子不同，但最后学习得到的参数矩阵</font>**$ A $**<font style="color:#117CEE;">的头部奇异向量重叠了</font>**
+ $ \Delta W_q $**<font style="color:#117CEE;">相比</font>**$ \Delta W_v $**<font style="color:#117CEE;">似乎拥有一个更大的内在维度</font>**，因为两轮实验中$ \Delta W_q $展现出更多公共的奇异向量方向
+ 作为比较，第三张图是两个随机生成的 Gaussian 矩阵的子空间相似度，可以看到它们互相不共享任何奇异向量，子空间没有任何重叠

+ **问题 3：更新量$ \Delta W $与原参数 $ W $的关系是什么？**

微调时的更新量$ \Delta W $是否与$ W $高度相关，或者从数学子空间的角度上讲

+ $ \Delta W $**<font style="color:#DF2A3F;">是否已经包含在参数</font>**$ W $**<font style="color:#DF2A3F;">的主要的奇异向量中，以及</font>**
+ $ \Delta W $**<font style="color:#DF2A3F;">子空间对应的方向在</font>**$ W $**<font style="color:#DF2A3F;">中有多显著</font>**

这**可以帮助我们更好地理解基于 LLM 微调时，参数调整的底层机制是什么，即微调到底在微调什么？**

我们可以**通过**$ U^TWV^T $**将**$ W $**投影到**$ \Delta W $**的**$ r $**维子空间中，其中**$ U,V $**分别是**$ \Delta W $**的左右奇异向量矩阵**，然后我们对比范数$ \|U^TWV^T\|_F,\|W\|_F $，作为基准，实验生成一个随机矩阵，并用它的前$ r $个奇异向量组成的矩阵$ U,V $来计算$ U^TWV^T $，结果如下：

<img src="../source/Chap10/LoRA子空间和原参数空间的相似性.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

从表中可以得出一些结论：

+ **<font style="color:#DF2A3F;">相比随机矩阵而言，</font>**$ \Delta W $**与**$ W $**<font style="color:#DF2A3F;">有较强的相关性</font>**，这说明$ \Delta W $捕捉了一些已经存在于$ W $中的特征
+ $ \Delta W $**<font style="color:#DF2A3F;">只捕捉和增强那些</font>**$ W $**<font style="color:#DF2A3F;">中存在但未展露的信息，而不是重复</font>**$ W $**<font style="color:#DF2A3F;">头部的奇异向量</font>**
+ **<font style="color:#DF2A3F;">信息增强因子相对很大</font>**，在$ r=4 $时增强倍率达到$ 21.5\approx 6.91 / 0.32 $

LoRA 的上述实验表明，**在微调时，低秩更新量矩阵**$ \Delta W $**可能针对特定的下游任务提升了原始预训练模型中已经学习到，但没有被展现、被增强的重要特征**

为了进一步论证该观点，我们可以绘制$ \Delta W=BA $中矩阵$ A $和$ W $的子空间相似性，如下图所示：

<img src="../source/Chap10/LoRA子空间增强原矩阵中的信号.png" width=900 style="display: block; margin-left: auto; margin-right: auto;">

从图中可以得出结论：

+ $ \Delta W $**<font style="color:#DF2A3F;">不包含</font>**$ W $**<font style="color:#DF2A3F;">中的头部的奇异向量方向</font>**，因为$ \Delta W $的前 4 的奇异向量方向和$ W $中前 10% 的奇异向量方向的子空间相似度几乎不超过 0.2
+ **<font style="color:#DF2A3F;">实验说明</font>**$ \Delta W $**<font style="color:#DF2A3F;">包含的那些在</font>**$ W $**中未强调的“特定任务”方向**
+ 此外，当$ r $增大时，$ \Delta W $偏向于选择提升$ W $中已经增强的更多方向

----